> You know, Apple Computers started in a garage, and we are starting in a condo. So we already have a leg up on Apple. - Pam Beesly


Today started as peaceful morning in Scraton Pennsylvania. There has been a long time vacated position ever since Ryan was fired as VP.
Everyone from the office just learnt that Charles Miner, previously worked at Saticoy Steel, is joining Dunder Mifflin as new VP.

Let's add Charles to Dunder Mifflin's scim group.


In [22]:
# First we need to import the scim client and configuration
from sync_app.client import ClientConfiguration, SCIMClient

# scim client configuration require server URL and access token.
# The server URL is the base URL of the SCIM server.
server_url = "https://api.scim.dev/scim/v2/"
# The access token is the token that is used to authenticate the client to the SCIM server.
# You can get the access token from the website
# Visit https://scim.dev/apikey/ to get your token
auth_token = "ACCESS_CODE"


# Now we have everything we need to create a scim client configuration
scim_dev_client_config = ClientConfiguration(
    server_url=server_url, auth_token=auth_token
)

# Now we can create a scim client using the configuration
scim_dev_client = SCIMClient(scim_dev_client_config)

In [23]:
# This block of code will generate all the users and update it to remote SCIM server. You should not run this code more than once.
# Otherwise you might get group name already exist error.
# If you want, you could run this to delete the group.
#
from sync_app.client import Filter

group_id = scim_dev_client.get_groups(
    filter=Filter(filter='displayName eq "Dunder Mifflin"')
).to_dict()["Resources"][0]["id"]
scim_dev_client.delete_group(group_id)
#

from scim_helper import course_setup

# You can ignore this part for now, the crouse_setup function create new group and add bunch exist users to the group.
dunder_mifflin_group_id, user_ids = course_setup(auth_token)

In [24]:
# You can take a look at the group_id and user_ids
print("Group ID is the UUID for the group created by the course_setup helper function")
print(dunder_mifflin_group_id)
print(
    "It also create a nice dictionary of user_id to user_name for the users added to the group."
)
user_ids

Group ID is the UUID for the group created by the course_setup helper function
9b8a9bcf-3280-427f-9f7d-704b3fecfe10
It also create a nice dictionary of user_id to user_name for the users added to the group.


{'Micheal Scott': '9b8a9bd0-103b-41a1-8a0f-e6f86265f972',
 'Jim Halpert': '9b8a9bd0-f3b8-4fb2-b098-64b7937a1f16',
 'Dwight Schrute': '9b8a9bd1-d6b5-4511-bbc8-406a453b7484',
 'Pam Beesly': '9b8a9bd2-b0a6-4e46-90d7-7356c3d6bbfd',
 'Ryan Howard': '9b8a9bd3-892a-40bf-8993-1cf71fbc6053',
 'Andy Bernard': '9b8a9bd4-61c4-4c07-8e3a-20c5b7c69fbf',
 'Kevin Malone': '9b8a9bd5-41e0-4a46-8971-4010849ff014',
 'Oscar Martinez': '9b8a9bd6-25ad-4f3d-9748-d073186cda01',
 'Angela Martin': '9b8a9bd7-04da-4100-bbf5-f42c9bcb555a',
 'Stanley Hudson': '9b8a9bd7-e62c-4de8-9bc3-4ca406dc98e8',
 'Phyllis Vance': '9b8a9bd8-c18d-4c18-afad-c152ed30e525',
 'Meredith Palmer': '9b8a9bd9-9f4b-48b2-a413-0cc8fe27968f',
 'Creed Bratton': '9b8a9bda-7fb8-4680-b386-cbc0676509c4',
 'Kelly Kapoor': '9b8a9bdb-5c97-41c6-a403-78cfc60c8049',
 'Toby Flenderson': '9b8a9bdc-3802-46af-a56c-864cc0b52ead'}

Enough of buzy work, now let's create a new user object!

Here is what User object required!

```
    def __init__(
        self,
        meta: ResourceMeta,
        user_id: UUID,
        displayName: Optional[str],
        active: bool,
        userName: str,
        name: Optional[Name],
        emails: Optional[Iterable[Email]],
        groups: Optional[Iterable[ResourceRef]] = None,
    ):
```

We can go over them one by one.


The first thing we need is some meta data!
ResourceMeta does provide create_meta function to create THE meta

```
def create_meta(resource_type: ResourceType, location: str) -> "ResourceMeta":
```

It requires the resource type and the resource URL
There are five different type resource that exist

- User
- Group
- ServiceProviderConfig
- ResourceType
- Schema

Looks like the one we need right now is `User` because duh, we are creating a user object.

But what the hack is location?
Resource Location is the endpoint where a resource could be re-fetched from the same service prodiver.
Since we (Duo) is storing the user, the location would be `https://duo.com/Users/<uuid>`


In [25]:
from sync_app.scim.resource import ResourceMeta

# We would also need some uuid
from uuid import uuid5, NAMESPACE_X500

# uuid5 is used to generate a unique identifier from a name and a namespace
# We would use the NAMESPACE_X500 as the namespace
# So we could use DN as the name to generate the uuid

uuid = uuid5(NAMESPACE_X500, "uid=Charles.Miner,DC=dundermifflin,DC=com")

meta = ResourceMeta.create_meta(
    "User",
    "https://duo.local/Users/{}".format(
        str(uuid),
    ),
)

Looking back on User Object

```
    def __init__(
        self,
        meta: ResourceMeta,
        user_id: UUID,
        displayName: Optional[str],
        active: bool,
        userName: str,
        name: Optional[Name],
        emails: Optional[Iterable[Email]],
        groups: Optional[Iterable[ResourceRef]] = None,
    ):
```

We actually killed two bird with one stone since meta require uuid.
`DisplayName` is The name of the User, suitable for display to end-users. The name SHOULD be the full name of the User being described, if known. So it should be like `Charles Miner`
`active` is A Boolean value indicating the User's administrative status. So `True` should be it.
`userName` is Unique identifier for the User, typically used by the user to directly authenticate to the service provider. Each User MUST include a non-empty userName value. This identifier MUST be unique across the service provider's entire set of Users. REQUIRED. So `cminer` should be fine.

`Name` is a werid one, because it contain some sub attribute.
`Name` Object looks like this, it's all optional but we fill familyName and givenName easily.

```
@dataclass
class Name:
    formatted: Optional[str] = None
    familyName: Optional[str] = None
    givenName: Optional[str] = None
```

`emails` is also a object that contain subattribute.

```
@dataclass
class Email:
    value: Optional[str] = None
    type: Optional[str] = None
    primary: Optional[bool] = None
    display: Optional[str] = None
```

`value` is email address
`type` is for work/home/other/etc
`primary` is if the email is the preferred one.
`display` is the the thing make least sense, so we will skip that for now.

`groups` is use set the group for the user, for now we are going to use the exist Dunder Mifflin Group!


In [26]:
# Now let's create a user!
from sync_app.scim.user import User, Name, Email
from sync_app.scim.resource import ResourceRef

cminer = User(
    meta=meta,
    user_id=uuid,
    displayName="Charles Miner",
    active=True,
    userName="cminer",
    name=Name(formatted="Charles Miner", familyName="Miner", givenName="Charles"),
    emails=[
        Email(
            value="CharlesMiner@dundermifflin.com",
            type="work",
            primary=True,
        )
    ],
    groups=[
        ResourceRef(
            ref="https://api.scim.dev/scim/v2/Group/{}".format(dunder_mifflin_group_id),
            value=dunder_mifflin_group_id,
            display="Dunder Mifflin",
        )
    ],
)

# Now we can try to push the user to the server!
response = scim_dev_client.create_user(cminer)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T22:30:27+00:00',
  'lastModified': '2024-03-11T22:30:27+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a9bdd-1461-4e33-960f-b8150799ca36'},
 'id': '9b8a9bdd-1461-4e33-960f-b8150799ca36',
 'active': True,
 'userName': 'cminer',
 'emails': [{'value': 'CharlesMiner@dundermifflin.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'CharlesMiner@dundermifflin.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': 'Charles Miner',
  'familyName': 'Miner',
  'givenName': 'Charles'}}

In [27]:
# We should also keep track of the newly created user to our user_ids dictionary
user_ids["Charles.Miner"] = response.to_dict()["id"]
user_ids

{'Micheal Scott': '9b8a9bd0-103b-41a1-8a0f-e6f86265f972',
 'Jim Halpert': '9b8a9bd0-f3b8-4fb2-b098-64b7937a1f16',
 'Dwight Schrute': '9b8a9bd1-d6b5-4511-bbc8-406a453b7484',
 'Pam Beesly': '9b8a9bd2-b0a6-4e46-90d7-7356c3d6bbfd',
 'Ryan Howard': '9b8a9bd3-892a-40bf-8993-1cf71fbc6053',
 'Andy Bernard': '9b8a9bd4-61c4-4c07-8e3a-20c5b7c69fbf',
 'Kevin Malone': '9b8a9bd5-41e0-4a46-8971-4010849ff014',
 'Oscar Martinez': '9b8a9bd6-25ad-4f3d-9748-d073186cda01',
 'Angela Martin': '9b8a9bd7-04da-4100-bbf5-f42c9bcb555a',
 'Stanley Hudson': '9b8a9bd7-e62c-4de8-9bc3-4ca406dc98e8',
 'Phyllis Vance': '9b8a9bd8-c18d-4c18-afad-c152ed30e525',
 'Meredith Palmer': '9b8a9bd9-9f4b-48b2-a413-0cc8fe27968f',
 'Creed Bratton': '9b8a9bda-7fb8-4680-b386-cbc0676509c4',
 'Kelly Kapoor': '9b8a9bdb-5c97-41c6-a403-78cfc60c8049',
 'Toby Flenderson': '9b8a9bdc-3802-46af-a56c-864cc0b52ead',
 'Charles.Miner': '9b8a9bdd-1461-4e33-960f-b8150799ca36'}

Now we have Charles in the system, Charles the new VP finally able to access his new laptop and all customer information for the first day!


Even Micheal had good time with Charles in the beginning, the tension between Micheal and Charles is so thich you can pratical cut it with a butter knife.

This result Micheal put his 2 week notice by the end of day. Micheal approved multiple Dunder Mifflin employee to join Micheal to his new paper company!


After goofy around for 2 week, the final day at Dunder Mifflin for Micheal has arrived. There is no one willing to quit the job at Dunder Mifflin and join Micheal for his new adventure.


Let's remove Micheal from the Dunder Mifflin group, so he can no long access inportant company secrets.


#### SCIM knowledge: _When you ask scim server to delete an user, scim server might not actual delete the user. But will not return the user information when you ask about the user again. So it might be a better idea to soft delete an user to make it inactive._


In [28]:
# Lets first deactive Micheal's status
from sync_app.scim.patchop import PatchOp, Operation

deactive_patch_op = PatchOp(
    operations=[Operation(op="replace", path="active", value=False)]
)

response = scim_dev_client.update_user(
    id=user_ids["Micheal Scott"], ops=deactive_patch_op
)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T22:30:18+00:00',
  'lastModified': '2024-03-11T22:30:28+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a9bd0-103b-41a1-8a0f-e6f86265f972'},
 'id': '9b8a9bd0-103b-41a1-8a0f-e6f86265f972',
 'active': False,
 'userName': 'mscott',
 'emails': [{'value': 'micheal.scott@dundermifflin.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'micheal.scott@dundermifflin.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Scott', 'givenName': 'Micheal'}}

In [29]:
# To remove user from group we need to make a patch call to the group API
# The API take a patch ops json object. So lets make that first
from sync_app.scim.patchop import PatchOp, Operation

remove_patch_op = PatchOp(
    operations=[Operation(op="remove", path="members", value=user_ids["Micheal Scott"])]
)

response = scim_dev_client.update_group(
    id=dunder_mifflin_group_id,
    ops=remove_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T22:30:18+00:00',
  'lastModified': '2024-03-11T22:30:18+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9bcf-3280-427f-9f7d-704b3fecfe10'},
 'id': '9b8a9bcf-3280-427f-9f7d-704b3fecfe10',
 'displayName': 'Dunder Mifflin',
 'name': 'Dunder Mifflin'}

> Confidence-- it's the food of the wise man, but the liquor of the fool. --Vikram


Oh no, Micheal Scott Paper Company just started.
We need to do some scim work to keep the scim record up to dates.
Here are list of things we need to.

- Create a new group called Micheal Scott Paper Company.
- Add Micheal, Pam, Ryan to the new Group.
- Deactive Pam from Dunder Mifflin group since she is no long working for Dunder Mifflin.


In [30]:
# Similar to before, this block of code is one time use only.
# If you want to run this code again, you should delete the group first.
#
from sync_app.client import Filter

group_id = scim_dev_client.get_groups(
    filter=Filter(filter='displayName eq "Micheal Scott Paper Company"')
).to_dict()["Resources"][0]["id"]
scim_dev_client.delete_group(group_id)


# Create a new group
from sync_app.scim.group import Group
from sync_app.scim.resource import ResourceMeta
from uuid import uuid5, NAMESPACE_X500

micheal_scott_paper_company = Group(
    displayName="Micheal Scott Paper Company",
    group_id=uuid5(NAMESPACE_X500, "cn=MichealScott,DC=PaperCompany,DC=com"),
    meta=ResourceMeta.create_meta(
        "Group",
        "https://duo.local/Groups/{}".format(
            uuid5(NAMESPACE_X500, "cn=MichealScott,DC=PaperCompany,DC=com")
        ),
    ),
)

response = scim_dev_client.create_group(micheal_scott_paper_company)

micheal_scott_paper_company_id = response.to_dict()["id"]

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T22:30:30+00:00',
  'lastModified': '2024-03-11T22:30:30+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9be1-6552-43b4-9786-0de6fd418d8a'},
 'id': '9b8a9be1-6552-43b4-9786-0de6fd418d8a',
 'displayName': 'Micheal Scott Paper Company',
 'name': 'Micheal Scott Paper Company'}

In [31]:
# Now let's try to do multiple operations in one patch call
# Let's create a patch operation to add all three people to the new group
from sync_app.scim.patchop import PatchOp, Operation

add_three_peope_op = PatchOp(
    operations=[
        Operation(
            op="add",
            path="members",
            value=[
                {"value": user_ids["Micheal Scott"]},
                {"value": user_ids["Pam Beesly"]},
                {"value": user_ids["Ryan Howard"]},
            ],
        )
    ]
)

response = scim_dev_client.update_group(
    id=micheal_scott_paper_company_id,
    ops=add_three_peope_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T22:30:30+00:00',
  'lastModified': '2024-03-11T22:30:30+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9be1-6552-43b4-9786-0de6fd418d8a'},
 'id': '9b8a9be1-6552-43b4-9786-0de6fd418d8a',
 'displayName': 'Micheal Scott Paper Company',
 'name': 'Micheal Scott Paper Company'}

In [32]:
# Now let's deactive Pam from dunder mifflin group
# We have done the similar thing for Micheal Scott before.
from sync_app.scim.patchop import PatchOp, Operation

remove_patch_op = PatchOp(
    operations=[
        Operation(op="remove", path="members", value=user_ids["Pam Beesly"]),
        Operation(op="remove", path="members", value=user_ids["Ryan Howard"]),
    ]
)

response = scim_dev_client.update_group(
    id=dunder_mifflin_group_id,
    ops=remove_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T22:30:18+00:00',
  'lastModified': '2024-03-11T22:30:18+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9bcf-3280-427f-9f7d-704b3fecfe10'},
 'id': '9b8a9bcf-3280-427f-9f7d-704b3fecfe10',
 'displayName': 'Dunder Mifflin',
 'name': 'Dunder Mifflin'}

Now we have created the group and populated some users in the group. Let's verify we have done everything correctly.
There are mutiple ways we can group information.


In [33]:
# The esiast way to get a group or user is to use the get_user or get_group method along with the id

# For example, this is how you get the original Dunder Mifflin group

response = scim_dev_client.get_group(micheal_scott_paper_company_id)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T22:30:30+00:00',
  'lastModified': '2024-03-11T22:30:30+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9be1-6552-43b4-9786-0de6fd418d8a'},
 'id': '9b8a9be1-6552-43b4-9786-0de6fd418d8a',
 'displayName': 'Micheal Scott Paper Company',
 'name': 'Micheal Scott Paper Company'}

In [34]:
# But it might be really hard to track the id of the user or group you want to get.
# Or you lost the id of the user or group you want to get.
# You could also use filter to filter the user or group you want to get.
# Now let's try to get the user we just created but with filter.

# First we need to create a new filter
from sync_app.client import Filter

filter = Filter(filter='displayName eq "Dunder Mifflin"')
response = scim_dev_client.get_groups(filter=filter)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:api:messages:2.0:ListResponse'],
 'totalResults': 1,
 'Resources': [{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
   'meta': {'resourceType': 'Group',
    'create': '2024-03-11T22:30:18+00:00',
    'lastModified': '2024-03-11T22:30:18+00:00',
    'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9bcf-3280-427f-9f7d-704b3fecfe10'},
   'id': '9b8a9bcf-3280-427f-9f7d-704b3fecfe10',
   'displayName': 'Dunder Mifflin',
   'name': 'Dunder Mifflin'}],
 'itemsPerPage': 1,
 'startIndex': 1}

In [35]:
# The response from get_groups does not contain all the information about the group.
# You can add attribute that you tell server on what attribute you want to get.
response = scim_dev_client.get_group(
    micheal_scott_paper_company_id, attributes=["members"]
)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T22:30:30+00:00',
  'lastModified': '2024-03-11T22:30:30+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a9be1-6552-43b4-9786-0de6fd418d8a'},
 'id': '9b8a9be1-6552-43b4-9786-0de6fd418d8a',
 'displayName': 'Micheal Scott Paper Company',
 'name': 'Micheal Scott Paper Company',
 'members': [{'$ref': 'https://api.scim.dev/scim/v2/Users/9b8a9bd0-103b-41a1-8a0f-e6f86265f972',
   'value': '9b8a9bd0-103b-41a1-8a0f-e6f86265f972',
   'display': 'mscott'},
  {'$ref': 'https://api.scim.dev/scim/v2/Users/9b8a9bd2-b0a6-4e46-90d7-7356c3d6bbfd',
   'value': '9b8a9bd2-b0a6-4e46-90d7-7356c3d6bbfd',
   'display': 'pbeesly'},
  {'$ref': 'https://api.scim.dev/scim/v2/Users/9b8a9bd3-892a-40bf-8993-1cf71fbc6053',
   'value': '9b8a9bd3-892a-40bf-8993-1cf71fbc6053',
   'display': 'rhoward'}]}

In [36]:
# You could also get the same result if you want to use user endpoint instead groups endpoint.

from sync_app.client import Filter

filter = Filter(filter='groups.value eq "{}"'.format(micheal_scott_paper_company_id))
response = scim_dev_client.get_users(filter=filter)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:api:messages:2.0:ListResponse'],
 'totalResults': 3,
 'Resources': [{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
   'meta': {'resourceType': 'User',
    'create': '2024-03-11T22:30:21+00:00',
    'lastModified': '2024-03-11T22:30:21+00:00',
    'location': 'https://api.scim.dev/scim/v2/Users/9b8a9bd3-892a-40bf-8993-1cf71fbc6053'},
   'id': '9b8a9bd3-892a-40bf-8993-1cf71fbc6053',
   'active': True,
   'userName': 'rhoward',
   'emails': [{'value': 'ryan.howard@dundermifflin.com',
     'type': 'other',
     'primary': True,
     'display': None},
    {'value': 'ryan.howard@dundermifflin.com',
     'type': 'work',
     'primary': True,
     'display': None}],
   'name': {'formatted': None, 'familyName': 'Howard', 'givenName': 'Ryan'}},
  {'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
   'meta': {'resourceType': 'User',
    'create': '2024-03-11T22:30:20+00:00',
    'lastModified': '2024-03-11T22:30:20+00:00',
    'location': '

#### SCIM knowledge: _You could query user attribute too!_


Looks like we also should update their emails! Let's do that next.


In [37]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    operations=[
        Operation(
            op="replace", path="emails.value", value="micheal.scott@michealscott.com"
        ),
    ]
)

response = scim_dev_client.update_user(
    id=user_ids["Micheal Scott"],
    ops=update_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T22:30:18+00:00',
  'lastModified': '2024-03-11T22:30:34+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a9bd0-103b-41a1-8a0f-e6f86265f972'},
 'id': '9b8a9bd0-103b-41a1-8a0f-e6f86265f972',
 'active': False,
 'userName': 'mscott',
 'emails': [{'value': 'micheal.scott@michealscott.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'micheal.scott@michealscott.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Scott', 'givenName': 'Micheal'}}

In [38]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    operations=[
        Operation(
            op="replace", path="emails.value", value="pam.beesly@michealscott.com"
        ),
    ]
)

response = scim_dev_client.update_user(
    id=user_ids["Pam Beesly"],
    ops=update_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T22:30:20+00:00',
  'lastModified': '2024-03-11T22:30:34+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a9bd2-b0a6-4e46-90d7-7356c3d6bbfd'},
 'id': '9b8a9bd2-b0a6-4e46-90d7-7356c3d6bbfd',
 'active': True,
 'userName': 'pbeesly',
 'emails': [{'value': 'pam.beesly@michealscott.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'pam.beesly@michealscott.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Beesly', 'givenName': 'Pam'}}

In [39]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    operations=[
        Operation(
            op="replace", path="emails.value", value="ryan.howard@michealscott.com"
        ),
    ]
)

response = scim_dev_client.update_user(
    id=user_ids["Ryan Howard"],
    ops=update_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T22:30:21+00:00',
  'lastModified': '2024-03-11T22:30:35+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a9bd3-892a-40bf-8993-1cf71fbc6053'},
 'id': '9b8a9bd3-892a-40bf-8993-1cf71fbc6053',
 'active': True,
 'userName': 'rhoward',
 'emails': [{'value': 'ryan.howard@michealscott.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'ryan.howard@michealscott.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Howard', 'givenName': 'Ryan'}}

#### Extra scim knowledges:

- You could also uses `PUT` calls to replace exist user and group. So instead of update one attribute, you will replace entire entry with new user/group. The main benefit for `PUT` call is you could save the same UUID.
- ... (more to come)


WooHoo! Now our job is done! Feel free to try more operition!
